In [1]:
from data_utils import *
import pandas as pd

In [2]:
data = get_CUSTOMDATA()

In [3]:
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
timestamps_train = data['timestamps_train']
timestamps_val = data['timestamps_val']
timestamps_test = data['timestamps_test']


print(X_train.shape)

(652845, 8)


In [4]:
print(X_train[0])
print(timestamps_train)

[ -3.9513943  -21.93701     -3.813674    -0.93178785  -6.88324
  -9.92183    -14.926612    -6.899644  ]
[1.73994478e+09 1.73994478e+09 1.73994478e+09 ... 1.73994805e+09
 1.73994805e+09 1.73994805e+09]


In [5]:
y_train = data['y_train']
y_train = np.argmax(y_train, axis=1)

y_val = data['y_val']
y_val = np.argmax(y_val, axis=1)

y_test = data['y_test']
y_test = np.argmax(y_test, axis=1)

print(y_train)

[3 3 3 ... 0 0 0]


In [6]:
print(y_train.shape)

(652845,)


In [7]:
train_df = pd.DataFrame(X_train, index=timestamps_train, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
train_df['target'] = y_train

val_df = pd.DataFrame(X_val, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
val_df['target'] = y_val

test_df = pd.DataFrame(X_test, columns=[f'emg_{i}' for i in range(X_train.shape[1])])
test_df['target'] = y_test

print('train')
print(train_df.head())
print('val')
print(val_df.head())
print('test')
print(test_df.head())


train
                  emg_0      emg_1      emg_2     emg_3    emg_4    emg_5  \
1.739945e+09  -3.951394 -21.937010  -3.813674 -0.931788 -6.88324 -9.92183   
1.739945e+09  -0.951394  44.062992  33.186325  0.068212  0.11676 -7.92183   
1.739945e+09  -4.951394  -7.937010 -27.813675 -0.931788  1.11676 -5.92183   
1.739945e+09   2.048606 -16.937010 -15.813674  4.068212  8.11676  4.07817   
1.739945e+09  13.048606 -55.937008 -24.813675 -6.931788 -1.88324 -3.92183   

                  emg_6     emg_7  target  
1.739945e+09 -14.926612 -6.899644       3  
1.739945e+09  -7.926612 -5.899644       3  
1.739945e+09   3.073388  1.100356       3  
1.739945e+09   5.073388  2.100356       3  
1.739945e+09  -2.926612 -1.899644       3  
val
      emg_0      emg_1     emg_2     emg_3    emg_4    emg_5     emg_6  \
0 -0.951394  -1.937010 -3.813674  0.068212  2.11676 -0.92183  0.073388   
1 -1.951394   6.062990 -3.813674 -2.931788  0.11676 -0.92183  0.073388   
2 -1.951394 -35.937008  3.186326  2.06821

In [ ]:
train_df.to_hdf('../data/custom_train.hdf5', key='df', mode='w')
val_df.to_hdf('../data/custom_val.hdf5', key='df', mode='w')
test_df.to_hdf('../data/custom_test.hdf5', key='df', mode='w')

In [ ]:
# Load them back and check
train_check = pd.read_hdf('../data/custom_train.hdf5', key='df')
val_check = pd.read_hdf('../data/custom_val.hdf5', key='df')
test_check = pd.read_hdf('../data/custom_test.hdf5', key='df')

print(f"Loaded train shape: {train_check.shape}")
print(f"Loaded val shape: {val_check.shape}")
print(f"Loaded test shape: {test_check.shape}")

In [ ]:
import h5py

# Path to your HDF5 file
hdf5_path = "../data/2021-07-22-1627004019-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f.hdf5"

def print_hdf5_structure(file_path):
    """Recursively prints the structure of an HDF5 file."""
    with h5py.File(file_path, "r") as f:
        def recursively_print(name, obj):
            if isinstance(obj, h5py.Dataset):
                print(f"Dataset: {name}, Shape: {obj.shape}, Dtype: {obj.dtype}")
            elif isinstance(obj, h5py.Group):
                print(f"Group: {name}")
                
        f.visititems(recursively_print)

def check_for_metadata(file_path):
    with h5py.File(file_path, "r") as f:
        # Check for attributes at the root level
        print("Root attributes:", list(f.attrs.items()))
        
        # Check for attributes in the emg2qwerty group
        if 'emg2qwerty' in f:
            print("emg2qwerty attributes:", list(f['emg2qwerty'].attrs.items()))
            
        # Check for attributes in the timeseries dataset
        if 'emg2qwerty/timeseries' in f:
            print("timeseries attributes:", list(f['emg2qwerty/timeseries'].attrs.items()))

check_for_metadata(hdf5_path)

print_hdf5_structure(hdf5_path)

In [ ]:
import h5py

hdf5_path = "../data/2021-07-22-1627004019-keystrokes-dca-study@1-0efbe614-9ae6-4131-9192-4398359b4f5f.hdf5"

with h5py.File(hdf5_path, "r") as f:
    dataset = f["emg2qwerty/timeseries"]  # Access the dataset
    
    # Print the first few rows
    for i in range(5):  # Print first 5 rows
        print(f"Row {i}: time={dataset[i]['time']}, emg_right={dataset[i]['emg_right']}, emg_left={dataset[i]['emg_left']}")

In [ ]:
import h5py
import numpy as np
import pandas as pd

def create_emg_hdf5_from_dataframe(output_file, df, finger_target_column='target', 
                                  condition='on_keyboard', daq_channels=8, 
                                  daq_sample_rate=200.0, duration_mins=None,
                                  finger_to_key_mapping=None):
    """
    Create an HDF5 file with the specified format using a pandas DataFrame as input.
    
    Parameters:
    -----------
    output_file : str
        Path to the output HDF5 file
    df : pandas DataFrame
        DataFrame containing EMG data and finger target information
    finger_target_column : str
        Column name in df that contains the finger/target information
    condition : str
        The experimental condition
    daq_channels : int
        Number of DAQ channels
    daq_sample_rate : float
        Sample rate of the DAQ in Hz
    duration_mins : float
        Duration of the recording in minutes (calculated if None)
    finger_to_key_mapping : dict or None
        Optional mapping from finger numbers to key characters
    """
    # Calculate duration if not provided
    if duration_mins is None and len(df) > 0:
        # Assuming the duration is based on sample count and sample rate
        duration_mins = len(df) / (daq_sample_rate * 60)
    
    # Default finger to key mapping if not provided
    if finger_to_key_mapping is None:
        finger_to_key_mapping = {
            0: 'spacebar', 1: 'j', 2: 'k', 3: 'l', 4: ';', 5: 'none'
        }
    
    # Generate keystrokes from finger movements
    keystrokes = []
    if finger_target_column in df.columns:
        # Find transitions in finger movements
        previous_finger = -1
        start_time = None
        
        # Create a timestamp column if it doesn't exist
        if 'timestamp' not in df.columns:
            # Create timestamps based on the sample rate
            df['timestamp'] = df.index / daq_sample_rate + 1627004023.0  # arbitrary base timestamp
            
        for idx, row in df.iterrows():
            current_finger = row[finger_target_column]
            
            # If moving from no movement to a finger movement, record start
            if previous_finger == -1 and current_finger != -1:
                start_time = row['timestamp']
            
            # If moving from a finger movement to a different state, record keystroke
            elif previous_finger != -1 and (current_finger == -1 or current_finger != previous_finger):
                if start_time is not None:
                    key = finger_to_key_mapping.get(previous_finger, 'unknown')
                    ascii_val = ord(key[0]) if len(key) == 1 else 0
                    keystrokes.append({
                        "ascii": ascii_val,
                        "key": key,
                        "start": start_time,
                        "end": row['timestamp']
                    })
                
                # If transitioning to a new finger, record new start time
                if current_finger != -1:
                    start_time = row['timestamp']
                else:
                    start_time = None
            
            previous_finger = current_finger
    
    # Format keystrokes as a string for the attribute
    keystrokes_str = str(keystrokes).replace("'", '"')
    
    # Convert DataFrame to structured numpy array
    # First, identify EMG columns (assuming they start with 'emg')
    emg_columns = [col for col in df.columns if col.startswith('emg')]
    
    
    if True:
        # If we have separate right and left EMG channels
        # Create dtype for structured array
        dtype = np.dtype([
            ('emg_right', np.float32, (len(emg_columns),)),
            ('emg_left', np.float32, (len(emg_columns),)),
            ('timestamp', np.float64),
            ('keyboard', np.uint8)
        ])
        
        # Create structured array
        emg_data = np.zeros(len(df), dtype=dtype)
        
        # Fill structured array
        for i, (_, row) in enumerate(df.iterrows()):
            emg_data[i]['emg_right'] = row[emg_columns].values.astype(np.float32)
            emg_data[i]['emg_left'] = 0.0
            emg_data[i]['timestamp'] = row['timestamp'] if 'timestamp' in row else (i / daq_sample_rate + 1627004023.0)
            emg_data[i]['keyboard'] = 1 if row[finger_target_column] != -1 else 0
    else:
        # If EMG channels are not separated into right/left, create a single EMG field
        dtype = np.dtype([
            ('emg', np.float32, (len(emg_columns),)),
            ('timestamp', np.float64),
            ('keyboard', np.uint8)
        ])
        
        # Create structured array
        emg_data = np.zeros(len(df), dtype=dtype)
        
        # Fill structured array
        for i, (_, row) in enumerate(df.iterrows()):
            emg_data[i]['emg'] = row[emg_columns].values.astype(np.float32)
            emg_data[i]['timestamp'] = row['timestamp'] if 'timestamp' in row else (i / daq_sample_rate + 1627004023.0)
            emg_data[i]['keyboard'] = 1 if row[finger_target_column] != -1 else 0
    
    # Create the HDF5 file
    with h5py.File(output_file, 'w') as f:
        # Create the main group
        group = f.create_group('emg2qwerty')
        
        # Set group attributes
        group.attrs['condition'] = condition
        group.attrs['daq_channels'] = str(daq_channels)
        group.attrs['daq_sample_rate'] = str(daq_sample_rate)
        group.attrs['duration_mins'] = duration_mins
        group.attrs['keystrokes'] = keystrokes_str
        
        # Create timeseries group
        ts_group = group.create_group('timeseries')
        
        # Create the dataset in the timeseries group
        if 'emg2qwerty/timeseries' in f:
            del f['emg2qwerty/timeseries']  # Delete the existing dataset
            dataset = f.create_dataset('emg2qwerty/timeseries', data=emg_data)
        else:
            dataset = f.create_dataset('emg2qwerty/timeseries', data=emg_data)
        
    return keystrokes

if __name__ == '__main__':
    create_emg_hdf5_from_dataframe("../data/custom_train.hdf5", train_df)

In [ ]:
import h5py
import numpy as np

def create_emg_hdf5(output_file, emg_data, keystrokes, condition='on_keyboard', 
                   daq_channels=16, daq_sample_rate=2000.0, duration_mins=None):
    """
    Create an HDF5 file with the specified format for EMG and keystroke data.
    
    Parameters:
    -----------
    output_file : str
        Path to the output HDF5 file
    emg_data : numpy structured array
        EMG data in a structured numpy array with the appropriate fields
    keystrokes : list
        List of keystroke dictionaries with 'ascii', 'key', 'start', and 'end' fields
    condition : str
        The experimental condition
    daq_channels : int
        Number of DAQ channels
    daq_sample_rate : float
        Sample rate of the DAQ in Hz
    duration_mins : float
        Duration of the recording in minutes (calculated if None)
    """
    # Calculate duration if not provided
    if duration_mins is None and len(emg_data) > 0:
        # Assuming the duration is based on sample count and sample rate
        duration_mins = len(emg_data) / (daq_sample_rate * 60)
    
    # Format keystrokes as a string for the attribute
    keystrokes_str = str(keystrokes).replace("'", '"')
    
    # Create the HDF5 file
    with h5py.File(output_file, 'w') as f:
        # Create the main group
        group = f.create_group('emg2qwerty')
        
        # Set group attributes
        group.attrs['condition'] = condition
        group.attrs['daq_channels'] = str(daq_channels)
        group.attrs['daq_sample_rate'] = str(daq_sample_rate)
        group.attrs['duration_mins'] = duration_mins
        group.attrs['keystrokes'] = keystrokes_str
        
        # Create timeseries group
        ts_group = group.create_group('timeseries')
        
        # Create the dataset in the timeseries group
        dataset = f.create_dataset('emg2qwerty/timeseries', data=emg_data)

def example_usage():
    """
    Example of how to use the create_emg_hdf5 function with sample data
    """
    # Define the structured dtype for the EMG data
    # Modify this to match your actual data structure
    dtype = np.dtype([
        ('emg_right', np.float32, (8,)),  # 8 channels for right hand EMG
        ('emg_left', np.float32, (8,)),   # 8 channels for left hand EMG
        ('timestamp', np.float64),         # timestamp
        ('keyboard', np.uint8)             # keyboard state
    ])
    
    # Create sample data (replace with your actual data)
    sample_count = X_train.shape[0]
    emg_data = np.zeros(sample_count, dtype=dtype)
    
    # Fill with random data for demonstration
    for i in range(sample_count):
        emg_data[i]['emg_right'] = np.random.random(8) * 0.01
        emg_data[i]['emg_left'] = np.random.random(8) * 0.01
        emg_data[i]['timestamp'] = 1627004023.0 + (i / 2000.0)
        emg_data[i]['keyboard'] = 0
    
    # Sample keystrokes (replace with your actual keystroke data)
    keystrokes = [
        {"ascii": 116, "key": "t", "start": 1627004023.3522873, "end": 1627004023.353288},
        {"ascii": 104, "key": "h", "start": 1627004023.353288, "end": 1627004023.4402883},
        {"ascii": 101, "key": "e", "start": 1627004023.4702878, "end": 1627004023.5752923}
    ]
    
    # Create the HDF5 file
    create_emg_hdf5('example_output.h5', emg_data, keystrokes)
    print("HDF5 file created successfully")

if __name__ == '__main__':
    # You can use the example or replace with your own data loading code
    example_usage()
    
    # For your actual use case, you would:
    # 1. Load your EMG data into a structured numpy array
    # 2. Load your keystroke data
    # 3. Call create_emg_hdf5 with your data